In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
from tqdm import tqdm
from facenet_pytorch import InceptionResnetV1, MTCNN
from demo_utils import *
from PIL import Image
import numpy  as np
import torch

In [3]:
images_dir = Path("/mnt/U/Datasets/lrs3pretrain/pretrain_images/")

In [4]:
speakers = list(images_dir.glob("*"))
utt_per_spk = {x.name: list(x.glob("*.jpg")) for x in speakers}

In [5]:
model = InceptionResnetV1(pretrained='vggface2', device="cuda").eval()
mtcnn = MTCNN(device="cuda")

In [14]:
speaker_names = []
image_paths = []
for speaker in speakers[:13]:
    im_paths =  utt_per_spk[speaker.name][:40]
    image_paths += im_paths
    speaker_names += [speaker.name]*len(im_paths)

In [15]:
images = list(map(Image.open, image_paths))
images_processed = list(map(mtcnn, tqdm(images, "Preprocessing images")))

Preprocessing images: 100%|██████████| 472/472 [00:08<00:00, 58.84it/s]


In [16]:
with torch.no_grad():
    embeds = np.array(list(map(lambda x: np.array(model(x.unsqueeze(0).cuda()).cpu()), tqdm(images_processed, "Getting embeddings"))))

Getting embeddings: 100%|██████████| 472/472 [00:06<00:00, 75.50it/s]


In [17]:
plot_projections(embeds[:,0], speaker_names, title="Embedding projections", n_components=3)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …